In [1]:
# ============================================================================
# Day 1: Synthetic Sensor Data Generation
# MSIM815 Final Project - Multi-Layer Sensor Fusion
# ============================================================================

"""
PURPOSE: Generate realistic synthetic sensor data for drone flight
OUTPUTS:
  - 3 sensor channels (IMU, GPS, Magnetometer)
  - 15-minute flight, 100Hz sampling
  - Ground truth trajectory
  - Training/validation/test splits
"""

# ============================================================================
# SETUP
# ============================================================================

from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os

# Paths
BASE_DIR = '/content/drive/MyDrive/MSIM815'
DATA_DIR = f'{BASE_DIR}/data/large_dataset'
os.makedirs(DATA_DIR, exist_ok=True)

print("✅ Setup complete")

# ============================================================================
# PARAMETERS
# ============================================================================

# Flight parameters
DURATION = 15 * 60  # 15 minutes in seconds
SAMPLE_RATE = 100   # Hz
NUM_SAMPLES = DURATION * SAMPLE_RATE  # 90,000 samples per channel

# Sensor noise parameters (realistic values)
NOISE_GPS = np.array([3.0, 3.0, 5.0])     # GPS: σ=[3m, 3m, 5m] (worse vertical)
NOISE_IMU = np.array([2.0, 2.0, 2.0])     # IMU: σ=2m (uniform)
NOISE_MAG = np.array([2.5, 2.5, 2.5])     # Magnetometer: σ=2.5m

# Trajectory parameters
SPEED = 10.0        # m/s
RADIUS = 30.0       # meters (for figure-8)

print(f"Generating {NUM_SAMPLES:,} samples per channel")
print(f"Duration: {DURATION}s ({DURATION/60:.1f} minutes)")

# ============================================================================
# GENERATE GROUND TRUTH TRAJECTORY (Figure-8)
# ============================================================================

def generate_figure8_trajectory(num_samples, speed=10.0, radius=30.0, sample_rate=100):
    """
    Generate 3D figure-8 trajectory

    Args:
        num_samples: Number of time steps
        speed: Linear speed (m/s)
        radius: Pattern radius (m)
        sample_rate: Hz

    Returns:
        positions: (N, 3) array of [x, y, z] positions
        velocities: (N, 3) array of [vx, vy, vz] velocities
    """
    t = np.linspace(0, num_samples/sample_rate, num_samples)

    # Angular frequency for figure-8
    omega = speed / radius

    # Figure-8 parametric equations
    x = radius * np.sin(omega * t)
    y = radius * np.sin(2 * omega * t) / 2
    z = 100 + 10 * np.sin(omega * t / 2)  # Varying altitude 90-110m

    positions = np.column_stack([x, y, z])

    # Compute velocities (numerical derivative)
    dt = 1.0 / sample_rate
    velocities = np.gradient(positions, dt, axis=0)

    return positions, velocities

print("\n🔄 Generating figure-8 trajectory...")
ground_truth_pos, ground_truth_vel = generate_figure8_trajectory(
    NUM_SAMPLES, SPEED, RADIUS, SAMPLE_RATE
)

print(f"   Position range: X[{ground_truth_pos[:,0].min():.1f}, {ground_truth_pos[:,0].max():.1f}]")
print(f"                   Y[{ground_truth_pos[:,1].min():.1f}, {ground_truth_pos[:,1].max():.1f}]")
print(f"                   Z[{ground_truth_pos[:,2].min():.1f}, {ground_truth_pos[:,2].max():.1f}]")

# ============================================================================
# GENERATE NOISY SENSOR MEASUREMENTS
# ============================================================================

def add_sensor_noise(ground_truth, noise_std, seed=None):
    """
    Add Gaussian noise to ground truth

    Args:
        ground_truth: (N, 3) clean positions
        noise_std: (3,) standard deviations [σx, σy, σz]
        seed: Random seed for reproducibility

    Returns:
        noisy_measurements: (N, 3) measurements with noise
    """
    if seed is not None:
        np.random.seed(seed)

    noise = np.random.randn(*ground_truth.shape) * noise_std
    return ground_truth + noise

print("\n📡 Generating sensor measurements...")

# Channel 1: IMU
np.random.seed(42)
channel_imu = add_sensor_noise(ground_truth_pos, NOISE_IMU, seed=42)
print(f"   IMU: σ={NOISE_IMU[0]:.1f}m")

# Channel 2: GPS
channel_gps = add_sensor_noise(ground_truth_pos, NOISE_GPS, seed=43)
print(f"   GPS: σ=[{NOISE_GPS[0]:.1f}, {NOISE_GPS[1]:.1f}, {NOISE_GPS[2]:.1f}]m")

# Channel 3: Magnetometer
channel_mag = add_sensor_noise(ground_truth_pos, NOISE_MAG, seed=44)
print(f"   Mag: σ={NOISE_MAG[0]:.1f}m")

# ============================================================================
# SAVE DATA
# ============================================================================

print("\n💾 Saving data...")

# Save as CSV
def save_channel(data, filename, prefix):
    df = pd.DataFrame(data, columns=['x', 'y', 'z'])
    filepath = f'{DATA_DIR}/{filename}'
    df.to_csv(filepath, index=False)
    print(f"   ✅ {prefix}: {filepath}")
    return filepath

# Save all channels
save_channel(channel_imu, 'channel_imu_15min.csv', 'IMU')
save_channel(channel_gps, 'channel_gps_15min.csv', 'GPS')
save_channel(channel_mag, 'channel_mag_15min.csv', 'Mag')

# Save ground truth
gt_df = pd.DataFrame(
    np.hstack([ground_truth_pos, ground_truth_vel]),
    columns=['x', 'y', 'z', 'vx', 'vy', 'vz']
)
gt_filepath = f'{DATA_DIR}/ground_truth_15min.csv'
gt_df.to_csv(gt_filepath, index=False)
print(f"   ✅ Ground truth: {gt_filepath}")

# ============================================================================
# VISUALIZE TRAJECTORY
# ============================================================================

print("\n📊 Creating visualization...")

fig = plt.figure(figsize=(16, 10))

# Plot 1: 3D Trajectory
ax1 = fig.add_subplot(2, 3, 1, projection='3d')
subsample = slice(0, NUM_SAMPLES, 100)  # Every 100th point
ax1.plot(ground_truth_pos[subsample, 0],
         ground_truth_pos[subsample, 1],
         ground_truth_pos[subsample, 2],
         'b-', linewidth=2, label='Ground Truth')
ax1.scatter(ground_truth_pos[0, 0], ground_truth_pos[0, 1], ground_truth_pos[0, 2],
           c='green', s=100, marker='o', label='Start')
ax1.scatter(ground_truth_pos[-1, 0], ground_truth_pos[-1, 1], ground_truth_pos[-1, 2],
           c='red', s=100, marker='x', label='End')
ax1.set_xlabel('X (m)')
ax1.set_ylabel('Y (m)')
ax1.set_zlabel('Z (m)')
ax1.set_title('3D Figure-8 Trajectory', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: XY plane (top-down view)
ax2 = fig.add_subplot(2, 3, 2)
ax2.plot(ground_truth_pos[:, 0], ground_truth_pos[:, 1], 'b-', alpha=0.6, linewidth=1)
ax2.scatter(ground_truth_pos[0, 0], ground_truth_pos[0, 1], c='green', s=100, marker='o')
ax2.scatter(ground_truth_pos[-1, 0], ground_truth_pos[-1, 1], c='red', s=100, marker='x')
ax2.set_xlabel('X (m)')
ax2.set_ylabel('Y (m)')
ax2.set_title('Top-Down View (XY Plane)', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axis('equal')

# Plot 3: Altitude profile
ax3 = fig.add_subplot(2, 3, 3)
time_axis = np.arange(NUM_SAMPLES) / SAMPLE_RATE / 60  # minutes
ax3.plot(time_axis, ground_truth_pos[:, 2], 'b-', linewidth=1.5)
ax3.set_xlabel('Time (minutes)')
ax3.set_ylabel('Altitude (m)')
ax3.set_title('Altitude Profile', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Plot 4: Sensor noise comparison
ax4 = fig.add_subplot(2, 3, 4)
sample_window = slice(0, 1000)  # First 10 seconds
ax4.plot(ground_truth_pos[sample_window, 0], 'k-', linewidth=2, label='Ground Truth')
ax4.plot(channel_imu[sample_window, 0], 'r-', alpha=0.6, label=f'IMU (σ={NOISE_IMU[0]:.1f}m)')
ax4.plot(channel_gps[sample_window, 0], 'g-', alpha=0.6, label=f'GPS (σ={NOISE_GPS[0]:.1f}m)')
ax4.plot(channel_mag[sample_window, 0], 'b-', alpha=0.6, label=f'Mag (σ={NOISE_MAG[0]:.1f}m)')
ax4.set_xlabel('Sample')
ax4.set_ylabel('X Position (m)')
ax4.set_title('Sensor Noise Comparison (First 10s)', fontweight='bold')
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3)

# Plot 5: Error histograms
ax5 = fig.add_subplot(2, 3, 5)
imu_error = np.linalg.norm(channel_imu - ground_truth_pos, axis=1)
gps_error = np.linalg.norm(channel_gps - ground_truth_pos, axis=1)
mag_error = np.linalg.norm(channel_mag - ground_truth_pos, axis=1)

ax5.hist(imu_error, bins=50, alpha=0.6, color='r', label='IMU', density=True)
ax5.hist(gps_error, bins=50, alpha=0.6, color='g', label='GPS', density=True)
ax5.hist(mag_error, bins=50, alpha=0.6, color='b', label='Mag', density=True)
ax5.set_xlabel('Position Error (m)')
ax5.set_ylabel('Density')
ax5.set_title('Sensor Error Distribution', fontweight='bold')
ax5.legend()
ax5.grid(True, alpha=0.3)

# Plot 6: Summary statistics
ax6 = fig.add_subplot(2, 3, 6)
ax6.axis('off')

summary_text = f"""
DATASET SUMMARY
{'='*40}

Flight Parameters:
  Duration:      {DURATION/60:.1f} minutes
  Sample Rate:   {SAMPLE_RATE} Hz
  Total Samples: {NUM_SAMPLES:,} per channel

Trajectory:
  Pattern:       Figure-8
  Speed:         {SPEED:.1f} m/s
  Radius:        {RADIUS:.1f} m
  Altitude:      {ground_truth_pos[:,2].min():.1f}-{ground_truth_pos[:,2].max():.1f} m

Sensor Noise (σ):
  IMU:           {NOISE_IMU[0]:.1f} m
  GPS (horiz):   {NOISE_GPS[0]:.1f} m
  GPS (vert):    {NOISE_GPS[2]:.1f} m
  Magnetometer:  {NOISE_MAG[0]:.1f} m

Error Statistics:
  IMU mean:      {np.mean(imu_error):.3f} m
  GPS mean:      {np.mean(gps_error):.3f} m
  Mag mean:      {np.mean(mag_error):.3f} m
"""

ax6.text(0.1, 0.5, summary_text, fontsize=10, family='monospace',
        verticalalignment='center')

plt.suptitle('Day 1: Synthetic Sensor Data Generation', fontsize=16, fontweight='bold')
plt.tight_layout()

viz_path = f'{DATA_DIR}/day1_data_visualization.png'
plt.savefig(viz_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"   ✅ Visualization: {viz_path}")

# ============================================================================
# GENERATE FFNN TRAINING DATA (Day 4 prep)
# ============================================================================

print("\n🔄 Generating FFNN training data...")

FFNN_DIR = f'{BASE_DIR}/data/ffnn_training'
os.makedirs(f'{FFNN_DIR}/train', exist_ok=True)
os.makedirs(f'{FFNN_DIR}/val', exist_ok=True)
os.makedirs(f'{FFNN_DIR}/test', exist_ok=True)

# Use GPS channel for FFNN
# Split: 60% train, 20% val, 20% test (temporal split)
n_train = int(0.6 * NUM_SAMPLES)
n_val = int(0.2 * NUM_SAMPLES)
n_test = NUM_SAMPLES - n_train - n_val

# Create EKF outputs (simulated - will be replaced by Day 3 real output)
# For now, use GPS with less noise as "EKF output"
ekf_noise = NOISE_GPS * 0.3  # EKF reduces noise by ~70%
ekf_outputs = add_sensor_noise(ground_truth_pos, ekf_noise, seed=45)

# Corrections = ground_truth - ekf_output
corrections = ground_truth_pos - ekf_outputs

# Split data
splits = {
    'train': (0, n_train),
    'val': (n_train, n_train + n_val),
    'test': (n_train + n_val, NUM_SAMPLES)
}

for split_name, (start, end) in splits.items():
    # EKF outputs (inputs to FFNN)
    ekf_split = ekf_outputs[start:end]
    pd.DataFrame(ekf_split, columns=['x', 'y', 'z']).to_csv(
        f'{FFNN_DIR}/{split_name}/ekf_outputs.csv', index=False
    )

    # Corrections (targets for FFNN)
    corr_split = corrections[start:end]
    pd.DataFrame(corr_split, columns=['dx', 'dy', 'dz']).to_csv(
        f'{FFNN_DIR}/{split_name}/corrections.csv', index=False
    )

    print(f"   ✅ {split_name}: {end-start:,} samples")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*70)
print("✅ DAY 1 COMPLETE")
print("="*70)
print(f"\nGenerated Files:")
print(f"  📁 {DATA_DIR}/")
print(f"     ├─ channel_imu_15min.csv ({NUM_SAMPLES:,} samples)")
print(f"     ├─ channel_gps_15min.csv ({NUM_SAMPLES:,} samples)")
print(f"     ├─ channel_mag_15min.csv ({NUM_SAMPLES:,} samples)")
print(f"     ├─ ground_truth_15min.csv ({NUM_SAMPLES:,} samples)")
print(f"     └─ day1_data_visualization.png")
print(f"\n  📁 {FFNN_DIR}/")
print(f"     ├─ train/ ({n_train:,} samples)")
print(f"     ├─ val/ ({n_val:,} samples)")
print(f"     └─ test/ ({n_test:,} samples)")
print("\nReady for Day 2: Python EKF implementation")
print("="*70)

MessageError: Error: credential propagation was unsuccessful